In [ ]:
# Colab에 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
!pip install numpy pandas matplotlib scikit-learn tensorflow

In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/bab2min/corpus/master/sentiment/steam.txt", filename="ratings_total.txt")

In [ ]:
total_data = pd.read_table('ratings_total.txt', names=['ratings', 'reviews'])
print('전체 리뷰 개수 :',len(total_data)) # 전체 리뷰 개수 출력

In [ ]:
total_data[:5]

In [ ]:
total_data['label'] = np.select([total_data.ratings > 0], [1], default=0)
total_data[:5]

In [ ]:
total_data['ratings'].nunique(), total_data['reviews'].nunique(), total_data['label'].nunique()

In [ ]:
total_data.drop_duplicates(subset=['reviews'], inplace=True) # reviews 열에서 중복인 내용이 있다면 중복 제거
print('총 샘플의 수 :',len(total_data))

In [ ]:
print(total_data.isnull().values.any())

In [ ]:
train_data, test_data = train_test_split(total_data, test_size = 0.25, random_state = 42)
print('훈련용 리뷰의 개수 :', len(train_data))
print('테스트용 리뷰의 개수 :', len(test_data))

In [ ]:
train_data['label'].value_counts().plot(kind = 'bar')

In [ ]:
print(train_data.groupby('label').size().reset_index(name = 'count'))

In [ ]:
# 한글과 공백을 제외하고 모두 제거
train_data['reviews'] = train_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['reviews'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
test_data.drop_duplicates(subset = ['reviews'], inplace=True) # 중복 제거
test_data['reviews'] = test_data['reviews'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['reviews'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

In [ ]:
mecab = Mecab()
print(mecab.morphs('와 이런 것도 상품이라고 차라리 내가 만드는 게 나을 뻔'))

In [ ]:
stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', '임', '게']

In [ ]:
train_data['tokenized'] = train_data['reviews'].apply(mecab.morphs)
train_data['tokenized'] = train_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
test_data['tokenized'] = test_data['reviews'].apply(mecab.morphs)
test_data['tokenized'] = test_data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
negative_words = np.hstack(train_data[train_data.label == 0]['tokenized'].values)
positive_words = np.hstack(train_data[train_data.label == 1]['tokenized'].values)

In [ ]:
negative_word_count = Counter(negative_words)
print(negative_word_count.most_common(20))

In [ ]:
positive_word_count = Counter(positive_words)
print(positive_word_count.most_common(20))

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(10,5))
text_len = train_data[train_data['label']==1]['tokenized'].map(lambda x: len(x))
ax1.hist(text_len, color='red')
ax1.set_title('Positive Reviews')
ax1.set_xlabel('length of samples')
ax1.set_ylabel('number of samples')
print('긍정 리뷰의 평균 길이 :', np.mean(text_len))

text_len = train_data[train_data['label']==0]['tokenized'].map(lambda x: len(x))
ax2.hist(text_len, color='blue')
ax2.set_title('Negative Reviews')
fig.suptitle('Words in texts')
ax2.set_xlabel('length of samples')
ax2.set_ylabel('number of samples')
print('부정 리뷰의 평균 길이 :', np.mean(text_len))
plt.show()

In [ ]:
X_train = train_data['tokenized'].values
y_train = train_data['label'].values
X_test= test_data['tokenized'].values
y_test = test_data['label'].values

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
threshold = 2
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)


In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

In [ ]:
tokenizer = Tokenizer(vocab_size, oov_token = 'OOV')
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

In [ ]:
print(X_test[:3])

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))

In [ ]:
max_len = 80
below_threshold_len(max_len, X_train)

In [ ]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
from tensorflow.keras.layers import Embedding, Dense, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
def sentiment_predict(new_sentence):
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = mecab.morphs(new_sentence)
    new_sentence = [word for word in new_sentence if not word in stopwords]
    encoded = tokenizer.texts_to_sequences([new_sentence])
    pad_new = pad_sequences(encoded, maxlen=max_len)

    score = float(loaded_model.predict(pad_new))
    if score > 0.5:
        return "긍정" if score >= 0.6 else "중립"
    else:
        return "부정" if (1 - score) >= 0.6 else "중립"

In [ ]:
sentiment_predict('서로 의지하면서 오래오래 사겼음 좋겠습니다.!')

In [ ]:
sentiment_predict('당장 헤어져! 난 혼또니 인정못해!!')

In [ ]:
sentiment_predict('이래서 아이도루한테 현질하면 안됨 ㅋㅋㅋㅋ')

In [ ]:
# 데이터 로드
comments_df = pd.read_excel("/content/drive/MyDrive/개별연구_최종/YouTube_Comments.xlsx")

# 댓글 데이터에 대해 감정 분석 수행
results = []
for sentence in comments_df['Comment']:
    results.append(sentiment_predict(sentence))

comments_df["Sentiment"] = results

# 새로운 파일로 저장
comments_df.to_excel("/content/drive/MyDrive/개별연구_최종/Sentiment_RNN.xlsx", index=False)
print("감정 분석 결과가 'Sentiment_RNN.xlsx' 파일에 저장되었습니다.")

In [ ]:
# 비율 시각화 원그래프

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 나눔고딕 폰트 설치
!apt-get update -qq
!apt-get install fonts-nanum -qq
!fc-cache -fv

# 나눔고딕 폰트 설정
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font = fm.FontProperties(fname=font_path)

# matplotlib 폰트 캐시 리빌드
def set_nanum_font():
    font_dirs = ['/usr/share/fonts/truetype/nanum']
    font_files = fm.findSystemFonts(fontpaths=font_dirs)
    for font_file in font_files:
        fm.fontManager.addfont(font_file)
    plt.rcParams['font.family'] = 'NanumGothic'

set_nanum_font()

# 파일 로드
file_path = '/content/drive/MyDrive/개별연구_최종/Sentiment_RNN.xlsx'  
comments_df = pd.read_excel(file_path)

# Sentiment 열의 값에 따라 데이터의 개수 세기
sentiment_counts = comments_df['Sentiment'].value_counts()

# 시각화
plt.figure(figsize=(8, 8))
plt.pie(sentiment_counts, labels=sentiment_counts.index, autopct='%1.1f%%', startangle=90, colors=['lightblue', 'lightcoral', 'lightgreen'])
plt.title('GRU')
plt.axis('equal')  # 원형으로 그리기
plt.show()

In [ ]:
# 대표 댓글 추출 시각화

import pandas as pd
import matplotlib.pyplot as plt

# 파일 경로
file_path = '/content/drive/MyDrive/개별연구_최종/Sentiment_RNN.xlsx'

# 데이터 불러오기
comments_df = pd.read_excel(file_path)

# 긍정과 부정 감정만 필터링
filtered_comments_df = comments_df[comments_df['Sentiment'].isin(['긍정', '부정'])]

# 각 감정별로 대표 댓글 3개씩 추출
top_comments = filtered_comments_df.groupby('Sentiment').apply(lambda x: x.nlargest(3, 'Like Count')).reset_index(drop=True)
print(top_comments)

# 시각화
fig, ax = plt.subplots(figsize=(12, 8))

colors = {'긍정': 'lightgreen', '부정': 'lightcoral'}
top_comments['Color'] = top_comments['Sentiment'].map(colors)

for sentiment in top_comments['Sentiment'].unique():
    subset = top_comments[top_comments['Sentiment'] == sentiment]
    ax.barh(subset['Comment'], subset['Like Count'], color=subset['Color'], label=sentiment)

ax.set_xlabel('Like Count')
ax.set_title('Top 3 Comments by Like Count for Positive and Negative Sentiments')
ax.legend(title='Sentiment')
plt.show()

In [ ]:
# 상위 20 키워드 시각화

import pandas as pd
import matplotlib.pyplot as plt
from konlpy.tag import Mecab
from collections import Counter

# 파일 경로
file_path = '/content/drive/MyDrive/개별연구_최종/Sentiment_RNN.xlsx'

# 데이터 불러오기
comments_df = pd.read_excel(file_path)

# Mecab 형태소 분석기 로드
mecab = Mecab()

# 불용어 정의
stopwords = ['이', '그', '저', '것', '수', '들', '중', '등', '더', '때', '그것', '아', '거기', '고', '곳', '나', '왜', '어떤', '로', '거', '게', '애', '건', '듯', '설', '일', '걸', '데', '난']

# 모든 댓글을 형태소 분석하여 명사만 추출
tokens = []
for comment in comments_df['Comment']:
    nouns = mecab.nouns(comment)
    filtered_nouns = [noun for noun in nouns if noun not in stopwords]
    tokens.extend(filtered_nouns)

# 토큰 빈도수 계산
token_counts = Counter(tokens)

# 상위 20개 키워드 추출
top_20_keywords = token_counts.most_common(20)
print(top_20_keywords)

# 상위 20개 키워드 시각화
keywords, counts = zip(*top_20_keywords)
plt.figure(figsize=(12, 8))
plt.barh(keywords, counts, color='skyblue')
plt.xlabel('Frequency')
plt.title('Top 20 Keywords')
plt.gca().invert_yaxis()
plt.show()